In [13]:
# libs
import pandas as pd
from datetime import datetime

# stats R type shit 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from statsmodels.api import OLS
import statsmodels.api as sm

In [21]:
# today's date string for file namine automation
today_date_str = datetime.now().strftime("_%Y-%m-%d.csv")
print(today_date_str)

_2024-02-22.csv


In [22]:
# cocncat base file name with today's date string
filename = f"trading_view_raw_data/tv_screen_gap-up{today_date_str}"

# read in trading view raw data
trading_view_raw_df = pd.read_csv(filename)

print(f'{len(trading_view_raw_df.index)} U.S. stocks trading w/ > 0% pre-market price gap change')
trading_view_raw_df.head(5)

1055 U.S. stocks trading w/ > 0% pre-market price gap change


,Symbol,Description,Exchange,Market capitalization,Market capitalization - Currency,Price,Price - Currency,Post-market Change,Post-market Change - Currency,Post-market Change %,...,Relative Volume 1 week,Volatility 1 day,Volatility 1 week,Volatility 1 month,Volume Weighted Average Price 1 day,Price to earnings ratio,Sector,Beta 1 year,Beta 3 years,Beta 5 years
0,MSFT,Microsoft Corporation,NASDAQ,3058738938168.469,USD,411.65,USD,0.10,USD,0.024292,...,0.59291387,2.606652,1.227215,1.556596,411.016667,37.23081932,Technology Services,0.98372686,1.0329885,0.8991026
1,NVDA,NVIDIA Corporation,NASDAQ,1939888515768.8328,USD,785.38,USD,13.57,USD,1.727826,...,1.07419748,14.959580,4.545887,3.647568,771.110000,65.79650651,Electronic Technology,1.5339884,2.2734668,1.6840769
2,META,"Meta Platforms, Inc.",NASDAQ,1239342381287.5408,USD,486.13,USD,0.22,USD,0.045255,...,0.56521134,4.612846,2.052824,2.565580,484.060600,32.63427831,Technology Services,1.3388069,1.0570395,1.1544267
3,BRK.B,Berkshire Hathaway Inc. New,NYSE,901056721791.5522,USD,415.16,USD,0.53,USD,0.127662,...,0.61409232,1.705596,1.007919,1.268516,414.140000,11.80763532,Finance,0.66524845,0.86781585,0.8809025
4,LLY,Eli Lilly and Company,NYSE,730624806336.6229,USD,769.64,USD,1.36,USD,0.176706,...,0.73592147,3.862377,3.322416,2.750365,767.010000,135.38797122,Health Technology,0.2675119,0.4242667,0.31574118


END - END - END - IN PROGRESS - END - END - END END - END - END - IN PROGRESS - END - END - END END - END - END - IN PROGRESS - END - END - END

In [12]:
# first 10 rows df
trading_view_raw_df_testing = trading_view_raw_df.head(10)
# save to csv
trading_view_raw_df_testing.to_csv('trading_view_raw_data/tv_screen_gap-up_testing.csv', index=False)
# make sure data is there at least
print(len(trading_view_raw_df_testing.index))

10


In [17]:
# baseline raw cols from trading view initial filter
features_raw_all = trading_view_raw_df.columns
# convert to list
features_raw_all = features_raw_all.tolist()
# look at the trading view columns
print(features_raw_all)

['Symbol', 'Description', 'Exchange', 'Market capitalization', 'Market capitalization - Currency', 'Price', 'Price - Currency', 'Post-market Change', 'Post-market Change - Currency', 'Post-market Change %', 'Post-market Close', 'Post-market Close - Currency', 'Float shares outstanding', 'Volume 1 day', 'Relative Volume 1 day', 'Average Volume 10 days', 'Post-market Volume', 'Relative Volume at Time', 'Relative Volume at Time.1', 'Volume 1 week', 'Relative Volume 1 week', 'Volatility 1 day', 'Volatility 1 week', 'Volatility 1 month', 'Volume Weighted Average Price 1 day', 'Price to earnings ratio', 'Sector', 'Beta 1 year', 'Beta 3 years', 'Beta 5 years']


In [20]:
# feature selection
# features = ['Market capitalization', 'Volume 1 day', 'Volatility 1 week', 'Beta 1 year', 'Sector']
features = features_raw_all
target = 'Post-market Change %'

# Preprocessing data: Drop rows with missing values in the selected columns
tv_clean_df = trading_view_raw_df[features + [target]].dropna()

# Check the unique values and their frequency for the target variable
print(tv_clean_df['Post-market Change %'].value_counts())


# # Encoding categorical variables and scaling numerical variables
# categorical_features = ['Sector']
# numeric_features = list(set(features) - set(categorical_features))

# preprocessor = ColumnTransformer(
#     transformers=[
#         ('num', StandardScaler(), numeric_features),
#         ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
#     ])

# # Prepare the data for modeling
# X = tv_clean_df[features]
# y = tv_clean_df[target]

# # Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Define and fit the model
# model = Pipeline(steps=[('preprocessor', preprocessor),
#                         ('regressor', LinearRegression())])

# model.fit(X_train, y_train)

# # Predict and evaluate the model
# y_pred = model.predict(X_test)
# mse = mean_squared_error(y_test, y_pred)
# r2 = r2_score(y_test, y_pred)

# print(f'MSE: {mse}')
# print(f'R^2: {r2}')

# # For detailed statistics, we need to fit the model using statsmodels
# # We'll prepare the data again because statsmodels requires it to be done differently
# X_train_transformed = model.named_steps['preprocessor'].transform(X_train)
# X_train_transformed = sm.add_constant(X_train_transformed)  # Adding a constant for the intercept

# # Fit the model using OLS
# ols_model = sm.OLS(y_train, X_train_transformed).fit()

# # Print the summary of the model to see detailed statistics
# print(ols_model.summary())


ValueError: Grouper for 'Post-market Change %' not 1-dimensional